### Import Functions

In [1]:
from ipynb.fs.full.utilities import *

### Data Prep

In [2]:
%%time
X_dev, y_dev = data_prep('../../data/Dev.txt', True)

2021-10-22 14:20:29 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |

2021-10-22 14:20:29 INFO: Use device: cpu
2021-10-22 14:20:29 INFO: Loading: tokenize
2021-10-22 14:20:29 INFO: Done loading processors!
2021-10-22 14:20:36 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |

2021-10-22 14:20:36 INFO: Use device: cpu
2021-10-22 14:20:36 INFO: Loading: tokenize
2021-10-22 14:20:36 INFO: Loading: pos
2021-10-22 14:20:37 INFO: Done loading processors!


CPU times: user 13min 54s, sys: 6min 31s, total: 20min 26s
Wall time: 14min 51s


In [3]:
verify_prepped_data(X_dev, y_dev)

In [4]:
X_dev[:1]

[[{'word': '^',
   'sent_len': 42,
   'upos': 'CCONJ',
   'first_word_in_sent': 'но',
   'contains_interrogative_word': False,
   'contains_interrogative_particle': False,
   'contains_imperative_verb': False,
   'contains_repetitive_conj_before': False,
   'between_two_verbs': False,
   'word_xpos_type': 'punct',
   'BOS': True,
   'next_word': 'но',
   'next_word_upos': 'CCONJ',
   'next_word_xpos_type': 'Cc',
   'word_after_next_word': 'синовете',
   'word_after_next_word_upos': 'NOUN',
   'word_after_next_word_xpos_type': 'Nc',
   'word_after_next_word_xpos_gender_number_article': 'mpd'},
  {'word': 'но',
   'sent_len': 42,
   'upos': 'CCONJ',
   'first_word_in_sent': 'но',
   'contains_interrogative_word': False,
   'contains_interrogative_particle': False,
   'contains_imperative_verb': False,
   'contains_repetitive_conj_before': False,
   'between_two_verbs': False,
   'word_xpos_type': 'Cc',
   'prev_word': '^',
   'prev_word_upos': 'CCONJ',
   'prev_word_xpos_type': 'punct',


In [5]:
%%time
X_train_60, y_train_60 = data_prep('../../data/Train_60.txt', True)

2021-10-22 14:41:25 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |

2021-10-22 14:41:25 INFO: Use device: cpu
2021-10-22 14:41:25 INFO: Loading: tokenize
2021-10-22 14:41:25 INFO: Done loading processors!
2021-10-22 14:42:05 INFO: Loading these models for language: bg (Bulgarian):
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |

2021-10-22 14:42:05 INFO: Use device: cpu
2021-10-22 14:42:05 INFO: Loading: tokenize
2021-10-22 14:42:05 INFO: Loading: pos
2021-10-22 14:42:06 INFO: Done loading processors!


CPU times: user 1h 25min, sys: 40min 13s, total: 2h 5min 14s
Wall time: 1h 30min 40s


In [6]:
verify_prepped_data(X_train_60, y_train_60)

In [7]:
X_train_60[:1]

[[{'word': '^',
   'sent_len': 6,
   'upos': 'PART',
   'first_word_in_sent': 'нека',
   'contains_interrogative_word': False,
   'contains_interrogative_particle': False,
   'contains_imperative_verb': False,
   'contains_repetitive_conj_before': False,
   'between_two_verbs': False,
   'word_xpos_type': 'Te',
   'BOS': True,
   'next_word': 'нека',
   'next_word_upos': 'PART',
   'next_word_xpos_type': 'Tv',
   'word_after_next_word': 'не',
   'word_after_next_word_upos': 'PART',
   'word_after_next_word_xpos_type': 'Tn'},
  {'word': 'нека',
   'sent_len': 6,
   'upos': 'PART',
   'first_word_in_sent': 'нека',
   'contains_interrogative_word': False,
   'contains_interrogative_particle': False,
   'contains_imperative_verb': False,
   'contains_repetitive_conj_before': False,
   'between_two_verbs': False,
   'word_xpos_type': 'Tv',
   'prev_word': '^',
   'prev_word_upos': 'PART',
   'prev_word_xpos_type': 'Te',
   'next_word': 'не',
   'next_word_upos': 'PART',
   'next_word_xpos_t

### CRF Model

In [8]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True,
    linesearch='MoreThuente',
    num_memories=8,
    period=12 
)
crf.fit(X_train_60, y_train_60)

CPU times: user 11min 22s, sys: 1.47 s, total: 11min 23s
Wall time: 11min 23s


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, linesearch='MoreThuente', max_iterations=200,
    num_memories=8, period=12)

In [9]:
labels = list(crf.classes_)
y_pred = crf.predict(X_dev)
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
labels.remove('')
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
punctuated_sentences = punctuate(X_dev, y_pred)
print(punctuated_sentences[:100])

0.9435366241653182


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['', ':', '!', ',', '?', ';', '.', '-', '"', ').', '(', ')', ':"', '".', '",', '),', '...'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

                  0.963     0.989     0.976    240418
           :      0.839     0.656     0.736       935
           !      0.579     0.236     0.335       373
           ,      0.796     0.672     0.729     18651
           ?      0.841     0.759     0.798       453
           ;      0.297     0.081     0.128       628
           .      0.943     0.986     0.964     11435
           -      0.511     0.128     0.205       935
           "      0.631     0.170     0.268      2243
          ).      0.955     0.934     0.944       271
           (      0.967     0.897     0.931      1693
           )      0.886     0.862     0.874      1003
          :"      1.000     0.103     0.187        68
          ".      0.518     0.097     0.163       300
          ",      0.738     0.591     0.656       848
          ),      0.828     0.688     0.752       385
         ...      0.286     0.029     0.052        70

    accuracy              

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=[':', '!', ',', '?', ';', '.', '-', '"', ').', '(', ')', ':"', '".', '",', '),', '...'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

           :      0.839     0.656     0.736       935
           !      0.579     0.236     0.335       373
           ,      0.796     0.672     0.729     18651
           ?      0.841     0.759     0.798       453
           ;      0.297     0.081     0.128       628
           .      0.943     0.986     0.964     11435
           -      0.511     0.128     0.205       935
           "      0.631     0.170     0.268      2243
          ).      0.955     0.934     0.944       271
           (      0.967     0.897     0.931      1693
           )      0.886     0.862     0.874      1003
          :"      1.000     0.103     0.187        68
          ".      0.518     0.097     0.163       300
          ",      0.738     0.591     0.656       848
          ),      0.828     0.688     0.752       385
         ...      0.286     0.029     0.052        70

   micro avg      0.851     0.716     0.778     40291
   macro avg      0.726   

#### Result: Better results for ":", '",'. Better precision for: "!", ".", "-", "...", ':"'. Better recall for: ",".
#### Increase in recall, decrease in precision: "?", "),". Worse precision for: ";".
#### Worse results for: '"' but significantly more examples (support), "(", ")", ").", '".'
#### Overall: slight decrease in performance.

### CRF Model - standard num_memories and period and less max_iterations

In [10]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    linesearch='MoreThuente',
    num_memories=6,
    period=10 
)
crf.fit(X_train_60, y_train_60)

CPU times: user 6min 46s, sys: 1.07 s, total: 6min 47s
Wall time: 6min 47s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, linesearch='MoreThuente', max_iterations=100,
    num_memories=6, period=10)

In [11]:
labels = list(crf.classes_)
y_pred = crf.predict(X_dev)
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
labels.remove('')
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
punctuated_sentences = punctuate(X_dev, y_pred)
print(punctuated_sentences[:100])

0.9430836072406606
              precision    recall  f1-score   support

                  0.962     0.989     0.976    240418
           :      0.848     0.645     0.733       935
           !      0.558     0.231     0.326       373
           ,      0.800     0.666     0.727     18651
           ?      0.849     0.722     0.780       453
           ;      0.305     0.068     0.112       628
           .      0.940     0.987     0.963     11435
           -      0.537     0.125     0.203       935
           "      0.616     0.165     0.261      2243
          ).      0.955     0.930     0.942       271
           (      0.970     0.891     0.929      1693
           )      0.888     0.863     0.876      1003
          :"      1.000     0.059     0.111        68
          ".      0.562     0.090     0.155       300
          ",      0.744     0.593     0.660       848
          ),      0.851     0.668     0.748       385
         ...      0.429     0.043     0.078        70

    acc

#### Result: Slightly lower recall and slightly greater precision. F-score almost the same.

### CRF - Model without verbs inbetween

In [15]:
X_dev_without_verbs_inbetween = [[{feature:word[feature] for feature in word if feature not in ('between_two_verbs')} for word in sentence] for sentence in X_dev]

In [16]:
X_train_60_without_verbs_inbetween = [[{feature:word[feature] for feature in word if feature not in ('between_two_verbs')} for word in sentence] for sentence in X_train_60]

In [17]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True,
    linesearch='MoreThuente',
    num_memories=8,
    period=12 
)
crf.fit(X_train_60_without_verbs_inbetween, y_train_60)

CPU times: user 11min 7s, sys: 1.26 s, total: 11min 8s
Wall time: 11min 8s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, linesearch='MoreThuente', max_iterations=200,
    num_memories=8, period=12)

In [19]:
labels = list(crf.classes_)
y_pred = crf.predict(X_dev_without_verbs_inbetween)
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
labels.remove('')
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
punctuated_sentences = punctuate(X_dev_without_verbs_inbetween, y_pred)
print(punctuated_sentences[:100])

0.9431354547417334
              precision    recall  f1-score   support

                  0.963     0.989     0.976    240418
           :      0.833     0.649     0.730       935
           !      0.574     0.228     0.326       373
           ,      0.798     0.668     0.727     18651
           ?      0.848     0.762     0.802       453
           ;      0.298     0.076     0.122       628
           .      0.942     0.987     0.964     11435
           -      0.500     0.125     0.200       935
           "      0.634     0.166     0.263      2243
          ).      0.958     0.937     0.948       271
           (      0.967     0.897     0.931      1693
           )      0.882     0.859     0.871      1003
          :"      0.889     0.118     0.208        68
          ".      0.577     0.100     0.170       300
          ",      0.733     0.587     0.652       848
          ),      0.827     0.683     0.748       385
         ...      0.286     0.029     0.052        70

    acc

#### Result: Results were better with verbs_inbetween

### CRF - Model without repetitive conjunction before

In [20]:
X_dev_without_rep_conj_before = [[{feature:word[feature] for feature in word if feature not in ('contains_repetitive_conj_before')} for word in sentence] for sentence in X_dev]

In [21]:
X_train_60_without_rep_conj_before = [[{feature:word[feature] for feature in word if feature not in ('contains_repetitive_conj_before')} for word in sentence] for sentence in X_train_60]

In [23]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True,
    linesearch='MoreThuente',
    num_memories=8,
    period=12 
)
crf.fit(X_train_60_without_rep_conj_before, y_train_60)

CPU times: user 11min, sys: 1.04 s, total: 11min 1s
Wall time: 11min 1s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, linesearch='MoreThuente', max_iterations=200,
    num_memories=8, period=12)

In [24]:
labels = list(crf.classes_)
y_pred = crf.predict(X_dev_without_rep_conj_before)
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
labels.remove('')
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
punctuated_sentences = punctuate(X_dev_without_rep_conj_before, y_pred)
print(punctuated_sentences[:100])

0.9434025613664837
              precision    recall  f1-score   support

                  0.963     0.989     0.976    240418
           :      0.834     0.651     0.732       935
           !      0.574     0.239     0.337       373
           ,      0.796     0.670     0.728     18651
           ?      0.841     0.773     0.806       453
           ;      0.300     0.076     0.122       628
           .      0.943     0.986     0.964     11435
           -      0.506     0.127     0.203       935
           "      0.628     0.171     0.268      2243
          ).      0.958     0.937     0.948       271
           (      0.966     0.898     0.931      1693
           )      0.885     0.861     0.873      1003
          :"      0.818     0.132     0.228        68
          ".      0.517     0.100     0.168       300
          ",      0.736     0.585     0.652       848
          ),      0.828     0.686     0.750       385
         ...      0.333     0.043     0.076        70

    acc

#### Result: Results are better with rep_conj_before

### CRF - Model on tags only

In [26]:
X_dev_tags_only = [[{feature:word[feature] for feature in word if feature not in ('word', 'prev_word', 'next_word', 'word_before_prev_word', 'word_after_next_word')} for word in sentence] for sentence in X_dev]

In [27]:
X_train_60_tags_only = [[{feature:word[feature] for feature in word if feature not in ('word', 'prev_word', 'next_word', 'word_before_prev_word', 'word_after_next_word')} for word in sentence] for sentence in X_train_60]

In [28]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True,
    linesearch='MoreThuente',
    num_memories=8,
    period=12 
)
crf.fit(X_train_60_tags_only, y_train_60)

CPU times: user 9min 5s, sys: 592 ms, total: 9min 6s
Wall time: 9min 5s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, linesearch='MoreThuente', max_iterations=200,
    num_memories=8, period=12)

In [30]:
labels = list(crf.classes_)
y_pred = crf.predict(X_dev_tags_only)
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
labels.remove('')
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
punctuated_sentences = punctuate(X_dev, y_pred)
print(punctuated_sentences[:100])

0.9218051705863994
              precision    recall  f1-score   support

                  0.945     0.989     0.967    240418
           :      0.676     0.335     0.448       935
           !      0.565     0.139     0.224       373
           ,      0.775     0.501     0.609     18651
           ?      0.830     0.764     0.795       453
           ;      0.377     0.032     0.059       628
           .      0.930     0.982     0.955     11435
           -      0.250     0.018     0.034       935
           "      0.193     0.008     0.015      2243
          ).      0.890     0.867     0.879       271
           (      0.844     0.773     0.807      1693
           )      0.757     0.750     0.754      1003
          :"      0.500     0.015     0.029        68
          ".      0.083     0.003     0.006       300
          ",      0.716     0.465     0.564       848
          ),      0.784     0.574     0.663       385
         ...      0.000     0.000     0.000        70

    acc

#### Result: Getting worse results

### CRF - Model with repetitive words instead of repetitive conjunctions

In [2]:
X_train_60 = load_json('../../data/Train_60_X.json')

In [3]:
y_train_60 = load_json('../../data/Train_60_y.json')

In [4]:
verify_prepped_data(X_train_60, y_train_60)

In [5]:
X_dev = load_json('../../data/Dev_X.json')

In [6]:
y_dev = load_json('../../data/Dev_y.json')

In [7]:
verify_prepped_data(X_dev, y_dev)

In [13]:
# input - sentence (list of dictionaries) and index of a word (dictionary)
# output - True/False depending on whether the sentence contains repetitive word before and the tag
#          of the word in the sentence (returns '' if False)
def contains_repetitive_word_before(sentence, i):
    for word_i in reversed(range(len(sentence[:i]))):
        if i < len(sentence)-1 and sentence[word_i]['word'].lower() == sentence[i+1]['word'].lower():
            return True, sentence[word_i]['word_xpos_type']
    
    return False, ''

In [14]:
X_train_60_rep_words = []

for sentence in X_train_60:
    new_sentence = []

    for i, word in enumerate(sentence):
        new_word = {}
        
        for feature in word:
            if feature == 'contains_repetitive_conj_before':
                pass
            else:
                new_word[feature] = word[feature]
        
        contains_rep_word, rep_word_tag = contains_repetitive_word_before(sentence, i)
        new_word.update({'contains_repetitive_word_before': contains_rep_word, 'repetitive_word_tag': rep_word_tag})
        new_sentence.append(new_word)
    
    X_train_60_rep_words.append(new_sentence)

In [15]:
X_train_60_rep_words[1]

[{'word': '^',
  'sent_len': 8,
  'upos': 'X',
  'first_word_in_sent': 'и',
  'contains_interrogative_word': True,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Lat',
  'BOS': True,
  'next_word': 'и',
  'next_word_upos': 'CCONJ',
  'next_word_xpos_type': 'Cp',
  'word_after_next_word': 'господ',
  'word_after_next_word_upos': 'PROPN',
  'word_after_next_word_xpos_type': 'Np',
  'word_after_next_word_xpos_gender_number_article': 'msi',
  'contains_repetitive_word_before': False,
  'repetitive_word_tag': ''},
 {'word': 'и',
  'sent_len': 8,
  'upos': 'CCONJ',
  'first_word_in_sent': 'и',
  'contains_interrogative_word': True,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Cp',
  'prev_word': '^',
  'prev_word_upos': 'X',
  'prev_word_xpos_type': 'Lat',
  'next_word': 'господ',
  'next_word_upos': 'PROPN',
  'next_word_

In [16]:
X_dev_rep_words = []

for sentence in X_dev:
    new_sentence = []

    for i, word in enumerate(sentence):
        new_word = {}
        
        for feature in word:
            if feature == 'contains_repetitive_conj_before':
                pass
            else:
                new_word[feature] = word[feature]
        
        contains_rep_word, rep_word_tag = contains_repetitive_word_before(sentence, i)
        new_word.update({'contains_repetitive_word_before': contains_rep_word, 'repetitive_word_tag': rep_word_tag})
        new_sentence.append(new_word)
    
    X_dev_rep_words.append(new_sentence)

In [17]:
X_dev_rep_words[1]

[{'word': '^',
  'sent_len': 32,
  'upos': 'PART',
  'first_word_in_sent': 'но',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': True,
  'between_two_verbs': False,
  'word_xpos_type': 'punct',
  'BOS': True,
  'next_word': 'но',
  'next_word_upos': 'CCONJ',
  'next_word_xpos_type': 'Cc',
  'word_after_next_word': 'когато',
  'word_after_next_word_upos': 'ADV',
  'word_after_next_word_xpos_type': 'Pr',
  'word_after_next_word_xpos_gender_number_article': '',
  'word_after_next_word_xpos_ref_type': 't',
  'contains_repetitive_word_before': False,
  'repetitive_word_tag': ''},
 {'word': 'но',
  'sent_len': 32,
  'upos': 'CCONJ',
  'first_word_in_sent': 'но',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': True,
  'between_two_verbs': False,
  'word_xpos_type': 'Cc',
  'prev_word': '^',
  'prev_word_upos': 'PART',
  'prev_word_xpos_type': 'punct',
  'next_wor

In [19]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True,
    linesearch='MoreThuente',
    num_memories=8,
    period=12 
)
crf.fit(X_train_60_rep_words, y_train_60)

CPU times: user 11min 48s, sys: 1.42 s, total: 11min 49s
Wall time: 11min 49s


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, linesearch='MoreThuente', max_iterations=200,
    num_memories=8, period=12)

In [20]:
labels = list(crf.classes_)
y_pred = crf.predict(X_dev_rep_words)
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
labels.remove('')
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
punctuated_sentences = punctuate(X_dev_rep_words, y_pred)
print(punctuated_sentences[:100])

0.9435560816039442


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['', ':', '!', ',', '?', ';', '.', '-', '"', ').', '(', ')', ':"', '".', '",', '),', '...'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

                  0.963     0.989     0.976    240418
           :      0.836     0.653     0.733       935
           !      0.591     0.236     0.337       373
           ,      0.794     0.674     0.729     18651
           ?      0.841     0.757     0.797       453
           ;      0.275     0.073     0.116       628
           .      0.943     0.987     0.964     11435
           -      0.502     0.129     0.206       935
           "      0.623     0.170     0.267      2243
          ).      0.955     0.937     0.946       271
           (      0.969     0.898     0.932      1693
           )      0.887     0.857     0.872      1003
          :"      0.750     0.088     0.158        68
          ".      0.536     0.100     0.169       300
          ",      0.734     0.591     0.654       848
          ),      0.824     0.691     0.751       385
         ...      0.286     0.029     0.052        70

    accuracy              

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=[':', '!', ',', '?', ';', '.', '-', '"', ').', '(', ')', ':"', '".', '",', '),', '...'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

           :      0.836     0.653     0.733       935
           !      0.591     0.236     0.337       373
           ,      0.794     0.674     0.729     18651
           ?      0.841     0.757     0.797       453
           ;      0.275     0.073     0.116       628
           .      0.943     0.987     0.964     11435
           -      0.502     0.129     0.206       935
           "      0.623     0.170     0.267      2243
          ).      0.955     0.937     0.946       271
           (      0.969     0.898     0.932      1693
           )      0.887     0.857     0.872      1003
          :"      0.750     0.088     0.158        68
          ".      0.536     0.100     0.169       300
          ",      0.734     0.591     0.654       848
          ),      0.824     0.691     0.751       385
         ...      0.286     0.029     0.052        70

   micro avg      0.850     0.717     0.778     40291
   macro avg      0.709   

#### Result: No great difference compared to the one with rep conj only. Even a bit of a loss.

### CRF - Model with feature for publicistic words

In [8]:
# input - sentence (list of dictionaries)
# output - True/False depending on whether the sentence contains publicistic word
def contains_publicistic_word(sentence):
    publicistic_words_list = ['казва', 'каза', 'заяви', 'заявява', 'отбеляза', 'добави', 'добавя',
                              'цитира', 'твърди', 'обясни', 'посочи', 'допълни', 'подчерта', 'писа',
                              'изтъкна', 'посочва', 'пише', 'заключи', 'сподели', 'обяснява', 'предупреди',
                              'отбелязва', 'предупреждава', 'призова', 'допълва', 'съобщи', 'заяви', 'обяви',
                              'коментира']

    for i in range(len(sentence)):
        if sentence[i]['word'] in publicistic_words_list:
            return True
    
    return False

In [9]:
X_train_60_pub_words = []

for sentence in X_train_60:
    new_sentence = []

    for i, word in enumerate(sentence):
        new_word = {}
        
        for feature in word:
            new_word[feature] = word[feature]
        
        contains_pub_word = contains_publicistic_word(sentence)
        new_word.update({'contains_publicistic_word': contains_pub_word})
        new_sentence.append(new_word)
    
    X_train_60_pub_words.append(new_sentence)

In [10]:
X_dev_pub_words = []

for sentence in X_dev:
    new_sentence = []

    for i, word in enumerate(sentence):
        new_word = {}
        
        for feature in word:
            new_word[feature] = word[feature]
        
        contains_pub_word = contains_publicistic_word(sentence)
        new_word.update({'contains_publicistic_word': contains_pub_word})
        new_sentence.append(new_word)
    
    X_dev_pub_words.append(new_sentence)

In [11]:
verify_prepped_data(X_dev_pub_words, y_dev)

In [12]:
verify_prepped_data(X_train_60_pub_words, y_train_60)

In [13]:
X_dev_pub_words[1]

[{'word': '^',
  'sent_len': 32,
  'upos': 'PART',
  'first_word_in_sent': 'но',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': True,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'punct',
  'BOS': True,
  'next_word': 'но',
  'next_word_upos': 'CCONJ',
  'next_word_xpos_type': 'Cc',
  'word_after_next_word': 'когато',
  'word_after_next_word_upos': 'ADV',
  'word_after_next_word_xpos_type': 'Pr',
  'word_after_next_word_xpos_gender_number_article': '',
  'word_after_next_word_xpos_ref_type': 't',
  'contains_publicistic_word': True},
 {'word': 'но',
  'sent_len': 32,
  'upos': 'CCONJ',
  'first_word_in_sent': 'но',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': True,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Cc',
  'prev_word': '^',
  'prev_word_upos': '

In [14]:
X_train_60_pub_words[1]

[{'word': '^',
  'sent_len': 8,
  'upos': 'X',
  'first_word_in_sent': 'и',
  'contains_interrogative_word': True,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Lat',
  'BOS': True,
  'next_word': 'и',
  'next_word_upos': 'CCONJ',
  'next_word_xpos_type': 'Cp',
  'word_after_next_word': 'господ',
  'word_after_next_word_upos': 'PROPN',
  'word_after_next_word_xpos_type': 'Np',
  'word_after_next_word_xpos_gender_number_article': 'msi',
  'contains_publicistic_word': True},
 {'word': 'и',
  'sent_len': 8,
  'upos': 'CCONJ',
  'first_word_in_sent': 'и',
  'contains_interrogative_word': True,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Cp',
  'prev_word': '^',
  'prev_word_upos': 'X',
  'prev_word_xpos_type': 'Lat',
  'next_word': 

In [15]:
save_as_json(X_train_60_pub_words, '../../data/Train_60_X.json')

In [16]:
save_as_json(X_dev_pub_words, '../../data/Dev_X.json')

In [28]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True,
    linesearch='MoreThuente',
    num_memories=8,
    period=12 
)
crf.fit(X_train_60_pub_words, y_train_60)

CPU times: user 12min 33s, sys: 1.85 s, total: 12min 34s
Wall time: 12min 34s


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, linesearch='MoreThuente', max_iterations=200,
    num_memories=8, period=12)

In [29]:
labels = list(crf.classes_)
y_pred = crf.predict(X_dev_pub_words)
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
labels.remove('')
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
punctuated_sentences = punctuate(X_dev_pub_words, y_pred)
print(punctuated_sentences[:100])

0.9438005848540651
              precision    recall  f1-score   support

                  0.963     0.988     0.976    240418
           :      0.838     0.653     0.734       935
           !      0.558     0.233     0.329       373
           ,      0.796     0.672     0.729     18651
           ?      0.845     0.773     0.807       453
           ;      0.293     0.084     0.131       628
           .      0.943     0.984     0.963     11435
           -      0.494     0.123     0.197       935
           "      0.618     0.196     0.298      2243
          ).      0.955     0.937     0.946       271
           (      0.968     0.897     0.931      1693
           )      0.887     0.865     0.876      1003
          :"      0.875     0.103     0.184        68
          ".      0.431     0.103     0.167       300
          ",      0.735     0.597     0.659       848
          ),      0.840     0.683     0.754       385
         ...      0.250     0.029     0.051        70

    acc

#### Result: Adding this feature is beneficial for the detection of quotation marks

### CRF - Model with standard parameters

In [15]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True,
    linesearch='MoreThuente',
    num_memories=6,
    period=10 
)
crf.fit(X_train_60_pub_words, y_train_60)

CPU times: user 11min 51s, sys: 1.22 s, total: 11min 52s
Wall time: 11min 52s


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, linesearch='MoreThuente', max_iterations=200,
    num_memories=6, period=10)

In [16]:
labels = list(crf.classes_)
y_pred = crf.predict(X_dev_pub_words)
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
labels.remove('')
print(metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print(metrics.flat_classification_report(y_dev, y_pred, labels=labels, digits=3))
punctuated_sentences = punctuate(X_dev_pub_words, y_pred)
print(punctuated_sentences[:100])

0.9437477001117717


/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=['', ':', '!', ',', '?', ';', '.', '-', '"', ').', '(', ')', ':"', '".', '",', '),', '...'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

                  0.963     0.988     0.976    240418
           :      0.834     0.648     0.729       935
           !      0.559     0.228     0.324       373
           ,      0.794     0.672     0.728     18651
           ?      0.841     0.770     0.804       453
           ;      0.302     0.083     0.130       628
           .      0.943     0.985     0.963     11435
           -      0.507     0.124     0.199       935
           "      0.635     0.199     0.303      2243
          ).      0.955     0.937     0.946       271
           (      0.967     0.897     0.931      1693
           )      0.884     0.862     0.873      1003
          :"      0.875     0.103     0.184        68
          ".      0.470     0.103     0.169       300
          ",      0.729     0.587     0.651       848
          ),      0.832     0.681     0.749       385
         ...      0.429     0.043     0.078        70

    accuracy              

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass labels=[':', '!', ',', '?', ';', '.', '-', '"', ').', '(', ')', ':"', '".', '",', '),', '...'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

           :      0.834     0.648     0.729       935
           !      0.559     0.228     0.324       373
           ,      0.794     0.672     0.728     18651
           ?      0.841     0.770     0.804       453
           ;      0.302     0.083     0.130       628
           .      0.943     0.985     0.963     11435
           -      0.507     0.124     0.199       935
           "      0.635     0.199     0.303      2243
          ).      0.955     0.937     0.946       271
           (      0.967     0.897     0.931      1693
           )      0.884     0.862     0.873      1003
          :"      0.875     0.103     0.184        68
          ".      0.470     0.103     0.169       300
          ",      0.729     0.587     0.651       848
          ),      0.832     0.681     0.749       385
         ...      0.429     0.043     0.078        70

   micro avg      0.849     0.717     0.778     40291
   macro avg      0.722   

#### Result: Not much of a difference